In [1]:
%matplotlib inline
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import h5py
from collections import Counter
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, AveragePooling1D
from keras import backend as K

Using TensorFlow backend.


In [2]:
from tools import open_file

### 0. parameters

In [3]:
num_classes = 3
num_dims = 4
epochs = 50
batch_size=10

### 1. Data loading and reconstruction

In [4]:
training_file = 'training set/trend_20day.h5'
file = open_file('h5py', training_file, mode='r')

In [5]:
X, Y = file['X'].value, file['Y'].value

In [6]:
Y = keras.utils.to_categorical(Y, num_classes)

In [7]:
#train data
length = int(len(X)*1)
x_train, x_test = X[:length], X[length:]
y_train, y_test = Y[:length], Y[length:]

#verification
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (863, 20, 4)
863 train samples
0 test samples


### 2. Construct Model

In [8]:
input_shape = X.shape[1:]

In [9]:
model = Sequential()
model.add(Conv1D(32, 3, input_shape=input_shape, activation='relu'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Dropout(0.25))
model.add(Conv1D(128, 3, activation='relu'))
#model.add(AveragePooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### 3. Train

In [14]:
model.fit(X, Y, batch_size=16, epochs=1, verbose=1)

Epoch 1/1
863/863 [==============================] - 1s - loss: 0.0089 - acc: 0.9977     


### 4. Accuracy

In [15]:
score = model.evaluate(x_test, y_test, verbose=1)

 32/151 [=====>........................] - ETA: 0s

In [16]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 3.22101695447e-05
Test accuracy: 1.0


### 5. Save to File

In [31]:
save_path = '../data/keras models/trend_10day.h5'

In [32]:
model.save(save_path)

In [33]:
model = keras.models.load_model(save_path)

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 18, 32)            416       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 16, 64)            6208      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 64)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 14, 128)           24704     
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 128)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1792)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               459008    
__________

### 6. Test Real Data

In [34]:
from IPython.display import clear_output
import time
from tools import ohlc_chart, norm, dataframe
from tools.factory import candle_chart

In [35]:
raw_file = open_file('h5py', 'raw data/daily_futures.h5', mode='r')

In [ ]:
window = 10
names = [name for name in raw_file]
while True:
    name = np.random.choice(names)
    raw_data = dataframe(name, raw_file).reset_index(drop=True)

    length = len(raw_data)
    
    
    for i in range(10):
        print(name)
        #시작일자 랜덤 설정
        start = np.random.choice(raw_data.index[:-window])
        end = start + window 
        data= raw_data.iloc[start:end]

        #Data Normalization
        base = np.abs(data.close.diff()).mean()
        data = norm(data)

        #그래프 
        fig, (ax) = plt.subplots(2,1, gridspec_kw = {'height_ratios':[1, 1]}, figsize=(12,14))
        fig.autofmt_xdate()

        s= start - 240
        ohlc_chart(ax[0], raw_data.iloc[s:end], colors=['k','k'], linewidth=1.44)
        ax[0].axvspan(start, end, facecolor='C1', alpha=0.2)

        candle_chart(ax[1], raw_data.iloc[start:end], colors=['red','blue'], linewidth=1.44)
        mean = raw_data[start:end].close.mean()
        ax[1].axhline(y=mean, linewidth=1.2, color='g')
        ymin = raw_data.open.loc[start]-base*2
        ymax = raw_data.open.loc[start]+base*2
        ax[1].axhspan(ymin, ymax, alpha=0.2, color='C1')
        ax[1].axhline(y=raw_data.open.loc[start], linewidth=1.2, color='red')
            
        ax[1].axvline(x= (start + end -1)/2, linewidth=1, color='g')
        plt.show()
        
        #추세
        time.sleep(0.1)
        
        #temp = data.values.T
        #test_data = np.stack( (temp[0::4, :], temp[1::4, :], temp[2::4, :], temp[3::4, :]), axis=-1)
        pred = model.predict(np.array([data[['open','high','low','close']].values]))
    
        print('pred:', np.argmax(pred[0]))
        print(np.round(pred[0],3))
        #print('prob: ', np.round(sess.run(tf.nn.softmax(pred)),3))
        key = input()
        
        clear_output(wait=True)
        if key == 'q':
            break
    if key == 'q':
            break
    clear_output(wait=True)

EUREX_FDAX


In [29]:
np.array([data.values]).shape

(1, 10, 6)